In [ ]:
import os
from pathlib import Path

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

## Set parameters in cell below, then run all:

In [ ]:
priority1 = [3737,3773,3774,3806,3807,3808,3809,3810,3841,3842,3843,3844,3845,3877,3878,3880,3911,3912,3913,3914,3944,3945,3946,3947,3948,3976,3978,3979,3980,4007,4008,4009,4010,4036]

### 1. BASIC CONFIG
home_dir = Path("~/data") ## Note this saves to your own homespace. use this for small files
scratch_dir = Path("/raid-cel/scratch/scratch-cel") ## Note scratch_dir has zero backup. This is good for temp files.
country = 'Paraguay'  ## ('Chile' | 'Paraguay')
data_source = 'stac' ##('stac' | 'GEE')
data_set = 'actual' ##('actual' | 'test')

test_grid = Path('/home/sandbox-cel/paraguay_lc/testing/ts_test_area_004057.gpkg')   #Only needed if data_set == 'test'
#crs = 'EqualEarth' ##('EqualEarth' | 'AlbersEqualArea' | 'UTM') note, this refers to the grid file being used - set automatically based on data_source
###cell pairs: Paraguay GEE940=stac3911
grid_cell = 3279
spec_index = 'evi2'
print_list=False
plot_yr = 2006
#grid_cells = priority1
grid_cells = [3737,3773,3806]

### 1b. params for single input plots (to view a single raw index or composite image)
image_type = 'AllRaw' ##(Sentinel'|'Landsat'|'Landsat5'|'Landsat7'|'Landsat8'|'Landsat9'|'AllRaw'|'Smooth'|'Smooth_old'|'Comp')
viewband = 'nir'
plot_day = 100
gamma = 2
interactive = True

### 1c. params for interactive plots
## use inputs for single input plots as well as:
# set above interactive==False
True
shpfile = 'poly'   ## ('point'| 'poly' | None)
get_new_coords = True
ptfile = 'interactive'  #path or 'interactive' if selection coordinates from map
#ptfile = Path("/home/sandbox-cel/paraguay_lc/vector/testpts_00305.csv")
polyfile = Path("/home/downspout-cel/paraguay_lc/vector/TestPolys/YerbaMatte_GEpolysEE.shp")
inputCRS = "epsg:8858"  ## esri:102033 = Albers SA Equal Area Conic, espg:8858 = Equal Earth

### 2. params for time series
load_prerunTS = False
smooth_TSfile = "/home/downspout-cel/chile_lc/OutputData/TSdfs/TS_evi2_smooth_2014-2018_natFor_1157.csv"
raw_TSfile = "/home/downspout-cel/chile_lc/OutputData/TSdfs/TS_evi2_rawSentinel_2014-2018_natFor_1157.csv"
                    
start_yr = 2020
end_yr = 2022
single_grid_cell = False
##For polygon-based sample:
#groundPolys = None   ##Polygon file if sample based on polygons (if = None: oldest, newest and npts are ignored):
ground_polys = Path("/home/downspout-cel/paraguay_lc/vector/TestPolys/YerbaMatte_GEpolysEE.shp")
oldest_samp = 0   ##oldest polygon to use (0 to ignore)
newest_samp = 0  ##newest polygon to use (0 to ignore)
npts = 1 ##pts sampled per polygon
seed1 = 88
##For point-based sample:
load_samp = False   ##if = False: PtFile is ignored

##3. Params for single output plots
class_prefix = ''
map_years = [2020,2021]

##6. MODELLING PARAMS (currently Random Forest) ##########################################################################
model_type = 'RF'
# main_model_dir should not be changed
main_model_dir = Path('/home/downspout-cel/{}_lc/classification'.format(country.lower()))
# local_model_dir can be changed to a local directory for testing/optimizing.
local_model_dir = Path('/home/downspout-cel/{}/classification/{}'.format(country.lower(),model_type))
#---------feature specs ------------------------------------------------------------------------------------------
feature_model = 'base6strat'
# if making new variable feature set (not just sample adjustments), set existing_feature_model to False
#     and sepcify spec_indices, si_vars and global_vars. If existing_eature_model == True, these variables will be ignored here.
existing_feature_model = True
spec_indices =["evi2","gcvi","wi","kndvi","nbr","ndmi"]
si_vars =["Max","Min","Amp","Avg","CV","Std","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
singleton_vars = ["forest_strata"]
include_polygon_data = False
poly_vars = []
samp_poly = 'D:/NasaProject/Paraguay/ClassificationModels/RF/pts_polyData.csv'
# ---------sample specs----------------------------------------------------------------------------------------------
sample_model = 'bal1000'
samp_pts = '/home/downspout-cel/paraguay_lc/vector/sampleData/SamplePts_Dec2023_ALL.csv'
# sample dataframe with variable data extracted to points. All points included (changes only if feature model changes of new points are made)
samp_pix_vars = '/home/downspout-cel/paraguay_lc/vector/ptsgdb_Dec18.csv'
# sample dataframe for current model (after sample adjustments are performed)
pixdf = '/home/downspout-cel/paraguay_lc/classification/RF/pixdf_base1000.csv'
#-----------model parameters-----------------------------------------------------------------------------------------
lc_mod = 'All' #'All'(=LC17) | 'crop_nocrop' | 'crop_nocrop_medcrop' | 'crop_nocrop_medcrop_tree' 
                 # | 'trans_cats' | 'veg'(=LC5) | 'cropType'(='LC_crops') | 'single_{}'
ranhold = 29
impmeth = 'Impurity'  # 'Impurity' | 'Permutation' | 'None'

### Do not need to modify below this line unless adding a new parameter
===============================================================================================================================

In [ ]:
if data_source == 'GEE':
    smooth_dir = Path("/home/downspout-cel/{}_lc/raster/grids".format(country.lower()))
    raw_dir = Path("/home/sandbox-cel/{}_lc/raster/grids".format(country.lower()))
    crs = 'Albers Equal Area'
    if country == 'Chile':
        grid_file = Path("/home/sandbox-cel/chile_lc/chl_grids.gpkg")
    elif country == 'Paraguay':
        grid_file = Path("/home/sandbox-cel/paraguay_lc/vector/pry_grids.gpkg")
    if data_set == 'test':
        smooth_dir = Path("/home/downspout-cel/{}_lc/testing/grids".format(country.lower()))
        raw_dir = Path("/home/sandbox-cel/{}_lc/testing/grid".format(country.lower()))
        grid_file = test_grid
elif data_source == 'stac':
    smooth_dir = Path("/home/downspout-cel/{}_lc/stac/grids".format(country.lower()))
    raw_dir = Path("/home/sandbox-cel/{}_lc/stac/grid".format(country.lower()))
    grid_file = Path("/home/sandbox-cel/LUCinLA_grid_8858.gpkg")
    crs = 'Equal Earth'
    if data_set == 'test':
        smooth_dir = Path("/home/downspout-cel/{}_lc/testing/grids".format(country.lower()))
        raw_dir = Path("/home/sandbox-cel/{}_lc/testing/grid".format(country.lower()))
        grid_file = test_grid
          
basic_config = { 'country' : country,
                'grid_cell' : grid_cell,
                'grid_cells' : grid_cells,
                'data_source' : data_source,
                'crs' : crs,
                'smooth_dir' : smooth_dir,
                'spec_index' : spec_index,
                'index_dir' : os.path.join(smooth_dir,'{:06d}'.format(grid_cell),'brdf_ts','ms',spec_index),
                'raw_dir' : raw_dir,
                'brdf_dir': os.path.join(raw_dir,'{:06d}'.format(grid_cell),'brdf'),
                'grid_file' : grid_file,
                'home_dir' : home_dir,
                'print_list' : print_list,
                'today' : today
              }

%store basic_config

if ptfile == 'interactive':
    ptfile = os.path.join(home_dir,'SelectedCoords.csv')
single_plot_params = {'image_type' : image_type,
                   'viewband' : viewband,
                   'ptfile' : ptfile,
                   'plot_day' : plot_day,
                   'plot_yr' : plot_yr,
                   'gamma': gamma,
                   'interactive' : interactive
              
    }

%store single_plot_params

interactive_plot_params = {'shpfile' : shpfile,
                         'ptfile' : ptfile,
                         'polyfile' : polyfile,
                         'inputCRS' : inputCRS,
                          'get_new_coords' : get_new_coords
                        }
%store interactive_plot_params

if single_grid_cell == True:
    grid_cells = [grid_cell]

timeseries_params = {'ptfile' : ptfile,
                    'start_yr': start_yr,
                    'end_yr': end_yr,
                    'image_type' : image_type,
                    'ground_polys': ground_polys,
                    'oldest_samp' : oldest_samp,
                    'newest_samp' : newest_samp,
                    'npts' : npts,
                    'seed1' : seed1,
                    'load_samp' : load_samp,
                    'load_prerunTS' : load_prerunTS,
                    'smooth_TSfile' : smooth_TSfile,
                    'raw_TSfile' : raw_TSfile
    }

%store timeseries_params


single_output_params = {'class_prefix' : class_prefix,
                     'class_dir': os.path.join(smooth_dir,'{:06d}'.format(grid_cell),'cls'),
                      'class_file':os.path.join(smooth_dir,'{:06d}'.format(grid_cell),'cls',class_prefix+'{:06d}'.format(grid_cell)+'.tif'),
                      'map_years': map_years
                      
    }

%store single_output_params

# setting feature model specs to None if model already exists in dictionary to avoid confusion (they will be looked up)
if existing_feature_model == True:
    spec_indices = None
    singleton_vars = None
    si_vars = None
classification_params = {'model_type' : model_type,
                         'main_model_dir': main_model_dir,
                         'local_model_dir' : local_model_dir,
                         'feature_model': feature_model,
                         'existing_feature_model': existing_feature_model,
                         'spec_indices' : spec_indices,
                         'si_vars' : si_vars,
                         'singleton_vars' : singleton_vars,
                         'samp_pts' : samp_pts,
                         'samp_pix_vars' : samp_pix_vars,
                         'pixdf' : pixdf,
                         'sample_model' : sample_model,
                         'model_name' : '{}_{}'.format(feature_model, sample_model),
                         'lc_mod' : lc_mod,
                         'ranhold' : ranhold,
                         'impmeth' : impmeth,
                         'samp_ploy' : samp_poly
                        }

%store classification_params